In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers
from augmentation import preprocess_image
from utils import *
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt
import numpy as np
import sys
from tqdm import tqdm

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
IMG_SIZE = 224
batch_size = 32
epochs = 45

In [ ]:
src_train_df, src_test_df, adp_train_df, adp_test_df, total_classes = create_DomainNet_data('clipart', 'real')

In [ ]:
train_label = list(src_train_df['label'])
train_img_path = list(src_train_df['path'])

test_label = list(src_test_df['label'])
test_img_path = list(src_test_df['path'])

train_total_samples = len(train_img_path)

In [ ]:
img_processor_train = preprocess_image(img_size = IMG_SIZE, color_distort = True, test_crop = False, is_training = True)
img_processor_test = preprocess_image(img_size = IMG_SIZE, color_distort = True, test_crop = False, is_training = False)

def process_img_train(image_path, label):
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    #print(image.shape)
#     #image = tf.image.convert_image_dtype(image, dtype=tf.float32)
#     image = tf.cast(image, tf.float32)
#     #image = image/255.
#     image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
#     #image = tf.clip_by_value(image, 0., 1.)
    image = img_processor_train(image)
    label_hot = tf.one_hot(label, total_classes)
    return image, label_hot
#     return image, label, label_hot

def process_img_test(image_path, label):
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    #print(image.shape)
#     image = tf.cast(image, tf.float32)
#     #image - image/255.
#     #image = tf.image.convert_image_dtype(image, dtype=tf.float32)
#     image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
#     #image = tf.clip_by_value(image, 0., 1.)
    image = img_processor_test(image)
    label_hot = tf.one_hot(label, total_classes)
    return image, label_hot
#     return image, label, label_hot

In [ ]:
# train_gen = ImageDataGenerator(preprocessing_function = img_processor_test)

# test_gen = ImageDataGenerator(preprocessing_function = img_processor_test)



# training_set=train_gen.flow_from_dataframe(dataframe=src_train_df,
#                                             directory=None,
#                                             x_col="path",
#                                             y_col="label",
#                                             batch_size=32,
#                                             seed=42,
#                                             shuffle=True,
#                                             class_mode="categorical",
#                                             target_size=(224, 224))


# test_set=test_gen.flow_from_dataframe(dataframe=src_test_df,
#                                             directory=None,
#                                             x_col="path",
#                                             y_col="label",
#                                             batch_size=32,
#                                             seed=42,
#                                             shuffle=False,
#                                             class_mode="categorical",
#                                             target_size=(224, 224))

# train_steps = training_set.n//training_set.batch_size
# validation_steps = test_set.n//test_set.batch_size

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_img_path, train_label))
test_ds = tf.data.Dataset.from_tensor_slices((test_img_path, test_label))


train_ds = (train_ds
            .map(process_img_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
            .shuffle(2048)
            .batch(batch_size, drop_remainder=True)
            .prefetch(tf.data.experimental.AUTOTUNE))

test_ds = (test_ds
           .map(process_img_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
           .batch(batch_size, drop_remainder=True)
           .prefetch(tf.data.experimental.AUTOTUNE))

In [ ]:
base_model = tf.keras.applications.efficientnet.EfficientNetB0(include_top=False,
                                                            weights='imagenet',
                                                            input_tensor=None,
                                                            input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                                            pooling='avg')

x = tf.keras.layers.Dropout(0.2)(base_model.layers[-1].output)
x = tf.keras.layers.Dense(total_classes, 
                          activation = 'softmax', 
                          kernel_initializer = 'glorot_uniform', 
                          bias_initializer = 'Zeros')(x)
model = tf.keras.models.Model(inputs = base_model.input, outputs = x)

In [ ]:
model.summary()

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01,
                                    momentum=0.9,
                                    nesterov=True,
                                    name='SGD')
#model = multi_gpu_model(model, gpus=4)
loss_function = tf.keras.losses.CategoricalCrossentropy()
model.compile(loss = loss_function,
optimizer = optimizer,
metrics = ['accuracy'])

In [ ]:
def scheduler(epoch):
    if epoch < 31:
        return 0.01
    elif epoch < 41:
        return 0.001
    elif epoch < 46:
        return 0.0001
    
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
callback_list = [callback]

In [ ]:
history = model.fit(
  train_ds,
  validation_data=test_ds,
  epochs=epochs, callbacks = callback_list
)

In [ ]:
logs = model.fit(training_set,steps_per_epoch=train_steps, epochs=100, 
                    validation_data=test_set,validation_steps=validation_steps, 
                    verbose = 1, callbacks = callback_list)

In [ ]:
history.history

In [ ]:
i = 0
for element in tqdm(test_ds):
    i += 1
    img = element[0]
    label = element[1]
    break

In [ ]:
img.shape

In [ ]:
img/255

In [ ]:
plt.imshow(img[1])

In [ ]:
label.shape

In [ ]:
label[1]

In [ ]:
(ds_train, ds_test), ds_info = tfds.load('mnist',
                                                 split=['train', 'test'],
                                                 shuffle_files=True,
                                                 as_supervised=True,
                                                 with_info=True)

In [ ]:
ds_train

In [ ]:
ds_train = (
    ds_train
    .map(process_img_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .shuffle(train_total_samples)
    .batch(batch_size, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [ ]:
len(ds_train)

In [ ]:
src_train_df.head(2)

In [ ]:
src_test_df.head(2)

In [ ]:
src_train_df[src_train_df['class'] == 'tiger']

In [ ]:
src_test_df[src_test_df['class'] == 'tiger']

In [ ]:
src_test_df[src_test_df['label'] == 27]

In [ ]:
src_train_df

In [ ]:
image_path = src_train_df['path'][0:4]
label = src_train_df['label'][0:4]
img = process_img_train(image_path, label)

In [ ]:
img

In [ ]:
image_path = src_train_df['path'][0:4]

In [ ]:
image_path